# Benchmarking Suite

In [1]:
from langchain_community.llms import Ollama
from factCheck import factCheckSingleClaim, factCheckSingleClaimNoContext

OLLAMA_HOST = "http://host.docker.internal:11434" # for when running within docker image
# OLLAMA_HOST = "http://localhost:11434"
 
ollama3 = Ollama(model="llama3", base_url=OLLAMA_HOST)
ollama32 = Ollama(model="llama3.2", base_url=OLLAMA_HOST)

claimCount = 100

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from sklearn.metrics import accuracy_score, f1_score

# Evaluation function
def evaluate_model(predictions, y):
    for idx in range(len(predictions)):
        if y[idx] in predictions[idx]:
            predictions[idx] = y[idx];
        else:
            predictions[idx] = predictions[idx][0];
    
    # Ensure predictions and dataset labels are aligned
    accuracy = accuracy_score(y, predictions)
    f1 = f1_score(y, predictions, average='weighted')
    return accuracy, f1

In [3]:
from benchmarking.AVERITEC.feverEval import get_fever_sample

dfFever = get_fever_sample(claimCount, './benchmarking/AVERITEC/data_train.json', 42)

In [ ]:
import time
from datetime import datetime
start_time = time.time()

feverMapping = {
    "true": ["Supported"],
    "mostly true": ["Supported", "Conflicting Evidence/Cherrypicking"],
    "mostly false": ["Refuted", "Conflicting Evidence/Cherrypicking"],
    "false": ["Refuted"],
    "not enough evidence": ["Not Enough Evidence", "Conflicting Evidence/Cherrypicking"]
}

modelResults = [{'name': 'FactCheck3'}, {'name': 'FactCheck32'}, {'name': 'Llama3'}, {'name': 'Llama32'}]
for modelIndex, model in enumerate([ollama3, ollama32]):
    # FactCheckLLM check
    origPred = []
    feverPred = []
    responses = []
    for row in dfFever.iterrows(): 
        pred = await factCheckSingleClaim(row[1]['claim'], model)
        origPred.append(pred['label'].lower())
        pred['label'] = feverMapping[pred['label'].lower()]
        feverPred.append(pred['label'])
        responses.append(pred['reply'])
        
    modelResults[modelIndex]['dataset'] = 'FEVER';
    modelResults[modelIndex]['original-prediction'] = origPred;
    modelResults[modelIndex]['mapped-predictions'] = feverPred;
    modelResults[modelIndex]['res'] = responses;
    
    acc, f1 = evaluate_model(feverPred, dfFever['label'].values)
    modelResults[modelIndex]['acc'] = acc
    modelResults[modelIndex]['f1'] = f1
    
    # base model check
    origPred = []
    feverPred = []
    responses = []
    for row in dfFever.iterrows(): 
        pred = await factCheckSingleClaimNoContext(row[1]['claim'], model)
        origPred.append(pred['label'].lower())
        pred['label'] = feverMapping[pred['label'].lower()]
        feverPred.append(pred['label'])
        responses.append(pred['reply'])
        
    modelResults[modelIndex+2]['dataset'] = 'FEVER';
    modelResults[modelIndex+2]['original-prediction'] = origPred;
    modelResults[modelIndex+2]['mapped-predictions'] = feverPred;
    modelResults[modelIndex]['res'] = responses;
    
    acc, f1 = evaluate_model(feverPred, dfFever['label'].values)
    modelResults[modelIndex+2]['acc'] = acc
    modelResults[modelIndex+2]['f1'] = f1

end_time = time.time()
elapsed_time = end_time - start_time
minutes, seconds = divmod(elapsed_time, 60)
print(f"{int(minutes)} minutes and {seconds:.2f} seconds")

CLAIM: Cyril Ramaphosa: South Africa has launched a Youth Employment Service, which places unemployed youth in paid internships in companies
Cache miss. Fetching data from website.
FactCheck Articles: []
News Articles: []
CLAIM: : A Muslim man in India was killed by a mob after being accused of cow slaughter.
Cache miss. Fetching data from website.
FactCheck Articles: []
News Articles: []
CLAIM: Deputy president William Ruto: The number of people on health insurance in Kenya increased from 3 million to 7 million in the period 2014 to 2019.
Cache miss. Fetching data from website.
FactCheck Articles: []
News Articles: []
CLAIM: Americans for Prosperity: The ExIm Bank provided taxpayer-backed handouts to foreign companies such as Air China and Saudi Aramco.”
Cache miss. Fetching data from website.
FactCheck Articles: []
News Articles: []
CLAIM: Andrew Mar: £22 billion was taken out of welfare spending in England between 2010 and 2015.
Cache miss. Fetching data from website.
FactCheck Arti

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Benchmark data
models =  [item["name"] for item in modelResults]
acc = [item["acc"] for item in modelResults]
f1 = [item["f1"] for item in modelResults]

# Bar chart parameters
x = np.arange(len(models))  # Position of bars
width = 0.4  # Width of bars

# Create the plot
fig, ax = plt.subplots()
bars = ax.bar(x, acc, width, color=['blue', 'green', 'purple', 'orange'])

# Add labels, title, and custom ticks
ax.set_xlabel('Models')
ax.set_ylabel('Acc. Scores')
ax.set_title('Comparison of Benchmark Accuracy')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_ylim(0, 1)  # Adjust y-axis range if needed
ax.bar_label(bars, fmt='%.2f', padding=3)  # Add values on top of bars

# Show the chart
plt.tight_layout()
plt.show()

In [ ]:
# Bar chart parameters
x = np.arange(len(models))  # Position of bars
width = 0.4  # Width of bars

# Create the plot
fig, ax = plt.subplots()
bars = ax.bar(x, f1, width, color=['blue', 'green', 'purple', 'orange'])

# Add labels, title, and custom ticks
ax.set_xlabel('Models')
ax.set_ylabel('F1 Scores')
ax.set_title('Comparison of Benchmark F1')
ax.set_xticks(x)
ax.set_xticklabels(models)
ax.set_ylim(0, 1)  # Adjust y-axis range if needed
ax.bar_label(bars, fmt='%.2f', padding=3)  # Add values on top of bars

# Show the chart
plt.tight_layout()
plt.show()